In [7]:
import numpy as np
import pandas as pd
from uuid import uuid4
from time import mktime
from random import choices
import ciso8601 as fasttime
from datetime import datetime, timedelta
from itertools import cycle, permutations, repeat

rng = np.random.default_rng()

## Status Reference Table (21 rows)

In [8]:
status_reference = [
    {"status_id": 0, "name": "image_pending", "category": "actionable_timed", "description": "Image under review."},
    {"status_id": 1, "name": "image_appealed", "category": "actionable_timed", "description": "Image to be reassessed."},
    {"status_id": 2, "name": "image_reported", "category": "actionable_untimed", "description": "Image reported, pending review."},
    {"status_id": 3, "name": "image_accepted", "category": "decision", "description": "Image approved for display."},
    {"status_id": 4, "name": "image_rejected", "category": "decision", "description": "Image rejected."},
    {"status_id": 5, "name": "image_marked_deletion", "category": "decision", "description": "Image marked for deletion, will automatically delete after 7 days."},
    {"status_id": 6, "name": "image_deleted", "category": "decision_final", "description": "Image deleted, do not display."},
    {"status_id": 7, "name": "user_acceptable", "category": "decision", "description": "User fine, no action neede."},
    {"status_id": 8, "name": "user_reported", "category": "actionable_untimed", "description": "User reported, pending review."},
    {"status_id": 9, "name": "user_muted", "category": "decision", "description": "User muted, cannot post comments or upload images."},
    {"status_id": 10, "name": "user_mute_appeal", "category": "actionable_untimed", "description": "User to be reassessed."},
    {"status_id": 11, "name": "user_banned", "category": "decision_final", "description": "User banned, account access restricted."},
    {"status_id": 12, "name": "comment_acceptable", "category": "decision", "description": "Comment fine, no action neede."},
    {"status_id": 13, "name": "comment_reported", "category": "actionable_untimed", "description": "Comment reported, pending review."},
    {"status_id": 14, "name": "comment_hidden", "category": "decision", "description": "Comment hidden from general view."},
    {"status_id": 15, "name": "comment_deleted", "category": "decision_final", "description": "Comment deleted, do not display."},
    {"status_id": 16, "name": "tag_pending", "category": "actionable_timed", "description": "Tag under review."},
    {"status_id": 17, "name": "tag_appealed", "category": "actionable_timed", "description": "Tag to be reassessed."},
    {"status_id": 18, "name": "tag_reported", "category": "actionable_untimed", "description": "Tag reported, pending review."},
    {"status_id": 19, "name": "tag_accepted", "category": "decision", "description": "Tag approved for use."},
    {"status_id": 20, "name": "tag_rejected", "category": "decision", "description": "Tag rejected for use."}
]

pd.DataFrame.from_records(status_reference).to_csv("./database/csv/status_reference.csv", index=False)
del status_reference

## Permission Reference Table (4 rows)

In [9]:
permission_reference = [
    {"permission_id": 0, "name": "basic", "description": "Basic user, no special permissions."},
    {"permission_id": 1, "name": "premium", "description": "Premium user, able to view deleted images."},
    {"permission_id": 2, "name": "moderator", "description": "Moderator, able to act on reports."},
    {"permission_id": 3, "name": "admin", "description": "Administrator."},
]

pd.DataFrame.from_records(permission_reference).to_csv("./database/csv/permission_reference.csv", index=False)
del permission_reference

## Tags Table (400 rows)

In [10]:
tag_names = [
    "Apple", "Air", "Conditioner", "Airport",
    "Ambulance", "Aircraft", "Apartment",
    "Arrow", "Antlers", "Apron", "Alligator",
    "Architect", "Ankle", "Armchair", "Aunt",
    "Ball", "Bermudas", "Beans", "Balloon",
    "Bear", "Blouse", "Bed", "Bow", "Bread",
    "Black", "Board", "Bones", "Bill",
    "Bitterness", "Boxers", "Belt", "Brain",
    "Buffalo", "Bird", "Baby", "Book", "Back",
    "Butter", "Bulb", "Buckles", "Bat", "Bank",
    "Bag", "Bra", "Boots", "Blazer", "Bikini",
    "Bookcase", "Bookstore", "Bus stop", "Brass",
    "Brother", "Boy", "Blender", "Bucket",
    "Bakery", "Bow", "Bridge", "Boat", "Car",
    "Cow", "Cap", "Cooker", "Cheeks", "Cheese",
    "Credenza", "Carpet", "Crow", "Crest",
    "Chest", "Chair", "Candy", "Cabinet", "Cat",
    "Coffee", "Children", "Cookware",
    "Chaise longue", "Chicken", "Casino",
    "Cabin", "Castle", "Church", "Cafe",
    "Cinema", "Choker", "Cravat", "Cane",
    "Costume", "Cardigan", "Chocolate", "Crib",
    "Couch", "Cello", "Cashier", "Composer",
    "Cave", "Country", "Computer", "Canoe",
    "Clock", "Dog", "Deer", "Donkey", "Desk",
    "Desktop", "Dress", "Dolphin", "Doctor",
    "Dentist", "Drum", "Dresser", "Designer",
    "Detective", "Daughter", "Egg", "Elephant",
    "Earrings", "Ears", "Eyes", "Estate",
    "Finger", "Fox", "Frock", "Frog", "Fan",
    "Freezer", "Fish", "Film", "Foot",
    "Flag", "Factory", "Father", "Farm",
    "Forest", "Flower", "Fruit", "Fork",
    "Grapes", "Goat", "Gown", "Garlic",
    "Ginger", "Giraffe", "Gauva", "Grains",
    "Gas station", "Garage", "Gloves",
    "Glasses", "Gift", "Galaxy", "Guitar",
    "Grandmother", "Grandfather", "Governor",
    "Girl", "Guest", "Hamburger", "Hand",
    "Head", "Hair", "Heart", "House", "Horse",
    "Hen", "Horn", "Hat", "Hammer", "Hostel",
    "Hospital", "Hotel", "Heels", "Herbs",
    "Host", "Jacket", "Jersey", "Jewelry",
    "Jaw", "Jumper", "Judge", "Juicer",
    "Keyboard", "Kid", "Kangaroo", "Koala",
    "Knife", "Lemon", "Lion", "Leggings",
    "Leg", "Laptop", "Library", "Lamb",
    "London", "Lips", "Lung", "Lighter",
    "Luggage", "Lamp", "Lawyer", "Mouse",
    "Monkey", "Mouth", "Mango", "Mobile",
    "Milk", "Music", "Mirror", "Musician",
    "Mother", "Man", "Model", "Mall",
    "Museum", "Market", "Moonlight",
    "Medicine", "Microscope", "Newspaper",
    "Nose", "Notebook", "Neck", "Noodles",
    "Nurse", "Necklace", "Noise", "Ocean",
    "Ostrich", "Oil", "Orange", "Onion",
    "Oven", "Owl", "Paper", "Panda",
    "Pants", "Palm", "Pasta", "Pumpkin",
    "Pharmacist", "Potato", "Parfume",
    "Panther", "Pad", "Pencil", "Pipe",
    "Police", "Pen", "Pharmacy",
    "Petrol station", "Police station",
    "Parrot", "Plane", "Pigeon", "Phone",
    "Peacock", "Pencil", "Pig", "Pouch",
    "Pagoda", "Pyramid", "Purse", "Pancake",
    "Popcorn", "Piano", "Physician",
    "Photographer", "Professor", "Painter",
    "Park", "Plant", "Parfume", "Radio",
    "Razor", "Ribs", "Rainbow", "Ring",
    "Rabbit", "Rice", "Refrigerator",
    "Remote", "Restaurant", "Road",
    "Surgeon", "Scale", "Shampoo", "Sink",
    "Salt", "Shark", "Sandals", "Shoulder",
    "Spoon", "Soap", "Sand", "Sheep",
    "Sari", "Stomach", "Stairs", "Soup",
    "Shoes",  "Scissors", "Sparrow",
    "Shirt", "Suitcase", "Stove",
    "Stairs", "Snowman", "Shower", "Swan",
    "Suit", "Sweater", "Smoke", "Skirt",
    "Sofa", "Socks", "Stadium", "Skyscraper",
    "School", "Sunglasses", "Sandals",
    "Slippers", "Shorts", "Sandwich",
    "Strawberry", "Spaghetti", "Shrimp",
    "Saxophone", "Sister", "Son", "Singer",
    "Senator", "Street", "Supermarket",
    "Swimming pool", "Star", "Sky", "Sun",
    "Spoon", "Ship", "Smile", "Table",
    "Turkey", "Tie", "Toes", "Truck",
    "Train", "Taxi", "Tiger", "Trousers",
    "Tongue", "Television", "Teacher",
    "Turtle", "Tablet", "Train station",
    "Toothpaste", "Tail", "Theater",
    "Trench coat", "Tea", "Tomato", "Teen",
    "Tunnel", "Temple", "Town", "Toothbrush",
    "Tree", "Toy", "Tissue", "Telephone",
    "Underwear", "Uncle", "Umbrella", "Vest",
    "Voice", "Veterinarian", "Villa", "Violin",
    "Village", "Vehicle", "Vase", "Wallet",
    "Wolf", "Waist", "Wrist", "Water melon",
    "Whale", "Water", "Wings", "Whisker",
    "Watch", "Woman", "Washing machine",
    "Wheelchair", "Waiter", "Wound",
    "Xylophone", "Zebra", "Zoo"
]

tag_count = len(tag_names)

description = cycle("")

tag_categories = cycle(["general"])
temp_status_id = rng.choice(
    a=[16, 17, 18, 19, 20],
    size=tag_count,
    p=[0.15, 0.01, 0.01, 0.71, 0.12]
)

creation_timestamps = np.random.randint(
    low=int(mktime(fasttime.parse_datetime("2009-05-14").timetuple())),
    high=int(mktime(fasttime.parse_datetime("2023-12-28").timetuple())),
    size=tag_count
)

tag_status = [
    19 if t < mktime(fasttime.parse_datetime("2021-04-03").timetuple()) else temp_status_id[idx]
    for idx, t in enumerate(creation_timestamps)
]

tags = sorted([*zip(range(tag_count), tag_categories, tag_names, description, tag_status, creation_timestamps)], key=lambda x: x[-1])
pd.DataFrame(tags, columns=["tag_id", "type_category", "name", "status_id", "creation_timestamps"]).to_csv("./database/csv/tags_table.csv", index=False)

del tag_names, tag_categories, temp_status_id, creation_timestamps, tag_status, tags

## Users Table (500,000 rows)

In [11]:
user_count = 500_000
string = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+_"
usernames = np.apply_along_axis(
    func1d=lambda x:"".join(x),
    axis=1,
    arr=rng.choice(a=list(string), size=(user_count, 14))
)

user_timestamps = np.sort(np.random.randint(
    low=int(mktime(fasttime.parse_datetime("2009-05-20").timetuple())),
    high=int(mktime(fasttime.parse_datetime("2023-12-28").timetuple())),
    size=user_count
))

temp_status_id = rng.choice(
    a=range(7, 12),
    size=user_count,
    p=[0.88, 0.01, 0.02, 0.01, 0.08]
)

status_id = [
    7 if t < mktime(fasttime.parse_datetime("2023-12-22").timetuple()) else temp_status_id[idx]
    for idx, t in enumerate(user_timestamps)
]

permission_level = rng.choice(
    a=range(4),
    size=user_count,
    p=[0.832388, 0.166522, 0.001064, 0.000026]
)


pd.DataFrame(
    zip(range(500_000), usernames, user_timestamps, status_id, permission_level),
    columns=["user_id", "username", "creation_timestamp", "status_id", "permission_id"]
).to_csv("./database/csv/users_table.csv")
del user_count, string, usernames, user_timestamps, temp_status_id, status_id, permission_level

## Images Table (100,000,000 rows)

In [12]:
batch_size = 100_000
batch_count = 100_000_000//batch_size

# Image Resolution constants
common_aspect_ratios = [(1, 1), (3, 2), (5, 4), (1, 2), (2, 1)]
custom_uncommon_ratios = [(h, w) for h,w in permutations(range(3, 11), r=2)]

image_sizes = ["800x600", "1080x1080", "1350x1080", "1280x720", "1240x1754", "1960x1080", "3840x2160"]
pixelart_heights = [64, 128, 256, 512]
custom_image_heights = [*range(300, 2050, 50)]

custom_common_image_sizes = []
for ratio in common_aspect_ratios:
    for height in custom_image_heights:
        custom_common_image_sizes.append(f"{ratio[0]*height}x{ratio[1]*height}")

custom_uncommon_image_sizes = []
for ratio in custom_uncommon_ratios:
    for height in custom_image_heights:
        custom_uncommon_image_sizes.append(f"{ratio[0]*height}x{ratio[1]*height}")

custom_pixelart_sizes = []
for ratio in common_aspect_ratios:
    for height in pixelart_heights:
        custom_pixelart_sizes.append(f"{ratio[0]*height}x{ratio[1]*height}")

# Newline padding
newline_padding = cycle("\n")
description = cycle("")

# Prewriting file header
with open("./database/csv/images_table.csv", "a+") as file:
    file.write("source_url,blob_storage_uuid,shape,upload_timestamp,upload_date,status_id,uploader_id,likes,dislikes")

    for idx in range(batch_count):
        index = range(batch_size*idx, batch_size*(idx+1))
        
        # Image URLs and UUIDs
        urls = np.array([
            f"https://www.{tup[0]}.{tup[1]}.{tup[2]}.{tup[3]}"
            for tup in rng.choice(a=list(range(256)), size=(batch_size, 4))
        ])

        blob_uuids = np.array([f"{uuid4().bytes}" for _ in range(batch_size)])

        # Image Resolutions
        image_shape = rng.choice(
            [
                *image_sizes,
                *custom_common_image_sizes,
                *custom_uncommon_image_sizes,
                *custom_pixelart_sizes,
            ],
            size=batch_size,
            p=[
                0.03, 0.06, 0.10, 0.06, 0.03, 0.39, 0.20,
                *repeat(0.08/len(custom_common_image_sizes), len(custom_common_image_sizes)),
                *repeat(0.04/len(custom_uncommon_image_sizes), len(custom_uncommon_image_sizes)),
                *repeat(0.01/len(custom_pixelart_sizes), len(custom_pixelart_sizes)),
            ]
        )

        # Dates and timestamps
        image_timestamps = np.sort(np.random.randint(
            low=int(mktime(fasttime.parse_datetime("2009-05-14").timetuple())),
            high=int(mktime(fasttime.parse_datetime("2023-12-28").timetuple())),
            size=batch_size
        ))

        image_dates = np.array([
            mktime(fasttime.parse_datetime(t).date().timetuple())
            for t in image_timestamps.astype("datetime64[s]").astype(str)
        ])

        # Status ID
        temp_status_id = rng.choice(
            a=range(7),
            size=batch_size,
            p=[0.05, 0.01, 0.02, 0.74, 0.03, 0.01, 0.14]
        )

        image_status = np.array([
            3 if t < int(mktime(fasttime.parse_datetime("2023-12-21").timetuple())) else temp_status_id[idx]
            for idx, t in enumerate(image_timestamps)
        ])

        # Uploader ID
        uploader_id = np.random.randint(low=0, high=500_000, size=batch_size)

        # Likes and Dislike
        likes = np.random.geometric(0.01, size=batch_size)
        dislikes = np.random.geometric(0.02, size=batch_size)

        file.writelines(
            (
                ",".join(row)
                for row
                in zip(
                    index, urls.astype(str), blob_uuids.astype(str),
                    image_shape.astype(str), image_timestamps.astype(str),
                    image_dates.astype(str), image_status.astype(str),
                    description, uploader_id.astype(str), likes.astype(str),
                    dislikes.astype(str), newline_padding
                )
            )
        )

del (
    batch_size, batch_count, common_aspect_ratios, custom_uncommon_ratios, image_sizes, pixelart_heights, custom_image_heights,
    custom_pixelart_sizes, custom_common_image_sizes, custom_uncommon_image_sizes, ratio, height, idx, urls, blob_uuids,
    image_shape, image_timestamps, image_dates, temp_status_id, image_status, uploader_id, likes, dislikes, newline_padding,
    description, index
)

## Comments Table (50,000,000 rows)

In [16]:
batch_size = 50_000
batch_count = 50_000_000//batch_size
string = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"

# Newline padding
newline_padding = cycle("\n")

with open("./database/csv/comments_table.csv", "a+") as file:
    file.write("status_id,content,creation_timestamp,edited,likes,dislikes")

    for idx in range(batch_count):
        index = np.arange(batch_size*idx, batch_size*(idx+1)).astype(str)

        comment_content = np.apply_along_axis(
            func1d=lambda x: " ".join(x),
            axis=1,
            arr=np.apply_along_axis(
                func1d=lambda x:"".join(x),
                axis=1,
                arr=rng.choice(a=list(string), size=(batch_size, 6, 8))
            )
        )

        comment_timestamps = np.sort(np.random.randint(
            low=int(mktime(fasttime.parse_datetime("2009-05-14").timetuple())),
            high=int(mktime(fasttime.parse_datetime("2023-12-28").timetuple())),
            size=batch_size
        ))

        temp_status_id = rng.choice(
            a=range(12, 16),
            size=batch_size,
            p=[0.97, 0.0015, 0.0085, 0.02]
        )

        image_status = np.array([
            12 if t < int(mktime(fasttime.parse_datetime("2023-12-21").timetuple())) else temp_status_id[idx]
            for idx, t in enumerate(comment_timestamps)
        ])

        comment_edited = np.random.randint(low=0, high=2, size=batch_size)
        comment_likes = np.random.geometric(0.1, size=batch_size)
        comment_dislikes = np.random.geometric(0.2, size=batch_size)

        file.writelines(
            (
                ",".join(row)
                for row
                in zip(
                    index,
                    temp_status_id.astype(str), comment_content.astype(str), comment_timestamps.astype(str),
                    comment_edited.astype(str), comment_likes.astype(str), comment_dislikes.astype(str),
                    newline_padding
                )
            )
        )

del (
    batch_size, batch_count, newline_padding, file, idx, temp_status_id,
    image_status, comment_content, comment_timestamps, comment_edited,
    comment_likes, comment_dislikes, index, string
)

## Image Tag Junction Table

In [ ]:
batch_size = 50_000
batch_count = 100_000_000//batch_size

